**Premier League 2020-2021**

Dados encontrados em: https://www.football-data.co.uk/englandm.php

Manipulação feita originalmente por mim, projeto livre.

In [ ]:
!pip install pandera

     |████████████████████████████████| 79 kB 3.1 MB/s 


In [ ]:
import pandas as pd
import pandera as pa

In [ ]:
df_original = pd.read_csv('/content/premierleague.csv', delimiter = ',', parse_dates=['Date'], dayfirst=True)
df_original

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,...,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,1,A,C Kavanagh,5,13,2,6,12,12,2,3,2,2,0,0,6.00,4.33,1.53,5.50,4.25,1.57,6.00,3.90,1.57,6.16,4.51,1.56,6.50,4.2,1.53,6.50,...,4.00,1.66,5.50,4.00,1.62,5.25,3.90,1.67,5.48,3.98,1.69,5.50,3.8,1.65,5.50,3.90,1.67,5.75,4.20,1.71,5.36,3.93,1.67,2.00,1.80,2.06,1.86,2.10,1.92,2.00,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,0,H,J Moss,5,9,3,5,14,11,7,3,2,1,0,0,3.10,3.25,2.37,3.00,3.20,2.45,3.15,2.95,2.40,3.32,3.29,2.40,3.20,3.2,2.35,3.20,...,3.25,2.40,3.00,3.30,2.40,3.05,2.90,2.45,3.09,3.27,2.54,3.10,3.1,2.45,3.10,3.25,2.45,3.25,3.33,2.55,3.08,3.22,2.47,2.20,1.66,2.26,1.72,2.27,1.78,2.18,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,2020-09-12,17:30,Liverpool,Leeds,4,3,H,3,2,H,M Oliver,22,6,6,3,9,6,9,0,1,0,0,0,1.28,6.00,9.50,1.26,6.25,10.50,1.35,5.00,8.50,1.31,6.25,9.92,1.27,6.0,10.00,1.30,...,6.00,11.00,1.25,6.25,11.00,1.30,6.00,9.00,1.28,6.34,11.38,1.25,6.0,12.00,1.29,6.00,11.50,1.30,6.75,12.27,1.28,6.16,10.63,1.50,2.62,1.51,2.76,1.53,2.82,1.50,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,2020-09-12,20:00,West Ham,Newcastle,0,2,A,0,0,D,S Attwell,15,15,3,2,13,7,8,7,2,2,0,0,2.15,3.40,3.40,2.15,3.40,3.40,2.15,3.15,3.40,2.18,3.61,3.50,2.15,3.5,3.40,2.15,...,3.60,3.75,1.95,3.70,3.75,2.05,3.25,3.75,2.04,3.59,3.92,2.00,3.5,3.80,2.00,3.50,3.90,2.07,3.78,3.99,2.01,3.57,3.79,1.90,1.90,2.00,1.92,2.00,2.05,1.91,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,2020-09-13,14:00,West Brom,Leicester,0,3,A,0,0,D,A Taylor,7,13,1,7,12,9,2,5,1,1,0,0,3.80,3.60,1.95,3.70,3.60,2.00,3.85,3.20,2.00,4.00,3.59,2.00,3.80,3.6,1.95,4.00,...,3.40,2.20,3.30,3.40,2.20,3.35,3.00,2.30,3.38,3.38,2.32,3.30,3.3,2.25,3.30,3.30,2.30,3.55,3.50,2.38,3.32,3.33,2.28,2.20,1.66,2.23,1.74,2.28,1.82,2.15,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,2,0,H,1,0,H,C Pawson,19,5,5,4,10,8,14,1,2,2,0,0,1.14,8.50,15.00,1.16,8.00,16.50,1.15,8.25,16.00,1.16,8.73,18.32,1.15,8.0,17.00,1.17,...,8.50,15.00,1.16,8.00,16.50,1.15,8.25,16.00,1.19,8.34,15.00,1.15,8.0,17.00,1.17,8.00,17.00,1.20,9.50,18.50,1.16,8.57,16.09,1.28,3.75,1.48,2.63,1.35,3.76,1.31,3.49,-2.25,1.86,2.04,1.88,2.03,1.98,2.14,1.88,2.00
376,E0,2021-05-23,16:00,Man City,Everton,5,0,H,2,0,H,M Oliver,21,8,11,3,8,10,7,5,2,2,0,0,1.44,5.00,6.50,1.44,5.00,6.50,1.47,4.90,6.00,1.43,5.09,7.24,1.44,4.8,7.00,1.45,...,6.00,9.00,1.30,6.00,9.00,1.30,6.25,8.50,1.28,6.28,10.34,1.30,5.8,9.50,1.30,5.75,9.00,1.32,6.65,11.25,1.30,6.12,9.60,1.44,2.75,1.53,2.63,1.50,2.96,1.45,2.77,-1.75,2.01,1.89,1.99,1.89,2.20,2.00,2.03,1.85
377,E0,2021-05-23,16:00,Sheffield United,Burnley,1,0,H,1,0,H,K Friend,12,10,3,3,11,1,8,9,3,1,0,0,2.80,3.60,2.40,2.80,3.60,2.40,2.85,3.60,2.40,2.87,3.74,2.43,2.80,3.5,2.45,2.88,...,3.40,2.50,2.75,3.50,2.50,2.75,3.40,2.55,2.85,3.45,2.58,2.88,3.5,2.40,2.80,3.50,2.50,3.08,3.67,2.63,2.80,3.47,2.53,1.80,2.00,1.86,2.03,1.89,2.17,1.79,2.05,0.00,2.04,1.86,2.05,1.86,2.17,1.90,2.03,1.84
378,E0,2021-05-23,16:00,West Ham,Southampton,3,0,H,2,0,H,M Atkinson,14,17,7,5,5,9,2,3,0,3,0,0,1.65,4.00,5.00,1.72,4.00,4.60,1.70,4.10,4.60,1.74,3.99,5.03,1.70,3.9,4.80,1.67,...,3.90,4.50,1.78,3.75,4.50,1.77,3.85,4.40,1

Como existem muitas colunas, irei criar um novo DF para trabalhar com colunas específicas.

In [ ]:
df = df_original.loc[:, ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'Referee', 'FTHG', 'FTAG']]
df

,Div,Date,Time,HomeTeam,AwayTeam,Referee,FTHG,FTAG
0,E0,2020-09-12,12:30,Fulham,Arsenal,C Kavanagh,0,3
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,J Moss,1,0
2,E0,2020-09-12,17:30,Liverpool,Leeds,M Oliver,4,3
3,E0,2020-09-12,20:00,West Ham,Newcastle,S Attwell,0,2
4,E0,2020-09-13,14:00,West Brom,Leicester,A Taylor,0,3
...,...,...,...,...,...,...,...,...
375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,C Pawson,2,0
376,E0,2021-05-23,16:00,Man City,Everton,M Oliver,5,0
377,E0,2021-05-23,16:00,Sheffield United,Burnley,K Friend,1,0
378,E0,2021-05-23,16:00,West Ham,Southampton,M Atkinson,3,0


*Iremos analisar se existem todos os dados ou se há dados ausentes*

In [ ]:
df.count()

Div         380
Date        380
Time        380
HomeTeam    380
AwayTeam    380
Referee     380
FTHG        380
FTAG        380
dtype: int64

In [ ]:
df.dtypes

Div                 object
Date        datetime64[ns]
Time                object
HomeTeam            object
AwayTeam            object
Referee             object
FTHG                 int64
FTAG                 int64
dtype: object

*Como a coluna 'Date' está do tipo objeto, iremos transformar no tipo data*

*(iremos fazer as modificações na leitura do csv)*

- df_original = pd.read_csv('/content/premierleague.csv', delimiter = ',',     parse_dates=['Date'], dayfirst=True)

In [ ]:
df.dtypes

Div                 object
Date        datetime64[ns]
Time                object
HomeTeam            object
AwayTeam            object
Referee             object
FTHG                 int64
FTAG                 int64
dtype: object

# **Etapa de Validação**

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        'Div': pa.Column(pa.String),
        'Date': pa.Column(pa.DateTime),
        'Time': pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5]?[0-9])?$')),
        'HomeTeam': pa.Column(pa.String),
        'AwayTeam': pa.Column(pa.String),
        'Referee': pa.Column(pa.String),
        'FTHG': pa.Column(pa.Int),
        'FTAG': pa.Column(pa.Int)
    }
)

In [ ]:
schema.validate(df)

,Div,Date,Time,HomeTeam,AwayTeam,Referee,FTHG,FTAG
0,E0,2020-09-12,12:30,Fulham,Arsenal,C Kavanagh,0,3
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,J Moss,1,0
2,E0,2020-09-12,17:30,Liverpool,Leeds,M Oliver,4,3
3,E0,2020-09-12,20:00,West Ham,Newcastle,S Attwell,0,2
4,E0,2020-09-13,14:00,West Brom,Leicester,A Taylor,0,3
...,...,...,...,...,...,...,...,...
375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,C Pawson,2,0
376,E0,2021-05-23,16:00,Man City,Everton,M Oliver,5,0
377,E0,2021-05-23,16:00,Sheffield United,Burnley,K Friend,1,0
378,E0,2021-05-23,16:00,West Ham,Southampton,M Atkinson,3,0


*Criando uma nova coluna -> Date + Time*

In [ ]:
df['Date_Time'] = pd.to_datetime(df.Date.astype(str) + ' ' + df.Time)
df

,Div,Date,Time,HomeTeam,AwayTeam,Referee,FTHG,FTAG,Date_Time
0,E0,2020-09-12,12:30,Fulham,Arsenal,C Kavanagh,0,3,2020-09-12 12:30:00
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,J Moss,1,0,2020-09-12 15:00:00
2,E0,2020-09-12,17:30,Liverpool,Leeds,M Oliver,4,3,2020-09-12 17:30:00
3,E0,2020-09-12,20:00,West Ham,Newcastle,S Attwell,0,2,2020-09-12 20:00:00
4,E0,2020-09-13,14:00,West Brom,Leicester,A Taylor,0,3,2020-09-13 14:00:00
...,...,...,...,...,...,...,...,...,...
375,E0,2021-05-23,16:00,Liverpool,Crystal Palace,C Pawson,2,0,2021-05-23 16:00:00
376,E0,2021-05-23,16:00,Man City,Everton,M Oliver,5,0,2021-05-23 16:00:00
377,E0,2021-05-23,16:00,Sheffield United,Burnley,K Friend,1,0,2021-05-23 16:00:00
378,E0,2021-05-23,16:00,West Ham,Southampton,M Atkinson,3,0,2021-05-23 16:00:00
